In [1]:
# HIDDEN
import warnings
# Ignore numpy dtype warnings. These warnings are caused by an interaction
# between numpy and Cython and can be safely ignored.
# Reference: https://stackoverflow.com/a/40846742
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import nbinteract as nbi

sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 7)
pd.set_option('display.max_columns', 8)
pd.set_option('precision', 2)
# This option stops scientific notation for pandas
# pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
# HIDDEN
def df_interact(df, nrows=7, ncols=7):
    '''
    Outputs sliders that show rows and columns of df
    '''
    def peek(row=0, col=0):
        return df.iloc[row:row + nrows, col:col + ncols]

    row_arg = (0, len(df), nrows) if len(df) > nrows else fixed(0)
    col_arg = ((0, len(df.columns), ncols)
               if len(df.columns) > ncols else fixed(0))
    
    interact(peek, row=row_arg, col=col_arg)
    print('({} rows, {} columns) total'.format(df.shape[0], df.shape[1]))

def display_df(df, rows=pd.options.display.max_rows,
               cols=pd.options.display.max_columns):
    with pd.option_context('display.max_rows', rows,
                           'display.max_columns', cols):
        display(df)

### Regularized Logistic Regression

As with linear regression, one common way of reducing the variance of the parameter estimator is to add a regularization term to the empirical risk objective. E.g.,

\begin{align*}
R(\beta, x, y, \lambda) &= - \frac{1}{n}\sum_{i=1}^n \left[ y_i x_i^T\beta + \log \sigma(-x_i^T\beta) \right] + \frac{1}{2} C \sum_{j=1}^J \beta_j^2 \\[10pt]
\nabla_{\beta} R(\beta, x, y, \lambda) &=  - \frac{1}{n}\sum_{i=1}^n \left(y_i - \sigma(x_i^T\beta)\right) x_i + C \beta \\[10pt]
\end{align*}

In [18]:
def regularized_logistic_regression(x, y, c):
    """Train a logistic regression classifier using gradient descent."""

    def l2_regularized_gradient(beta, x, y):
        return risk_gradient(beta, x, y) + c * beta

    beta0 = np.zeros(x.shape[0])
    beta = gradient_descent(x, y, beta0, l2_regularized_gradient)
    return beta    

def search_for_c(features):
    for c in 2.0 ** np.arange(-10, 10, 2):
        print("c =", c)
        beta = regularized_logistic_regression(features(train), y_train, c)
        print("sum(beta**2) = ", sum(beta**2))
        evaluate(beta, features)
        print()
        
search_for_c(all_features)

c = 0.0009765625
sum(beta**2) =  181.05862166469308
Train: 406/426 (95.3%)
Test: 137/143 (95.8%)

c = 0.00390625
sum(beta**2) =  153.17656348299207
Train: 406/426 (95.3%)
Test: 137/143 (95.8%)

c = 0.015625
sum(beta**2) =  84.76727676186307
Train: 401/426 (94.1%)
Test: 139/143 (97.2%)

c = 0.0625
sum(beta**2) =  16.86808437919907
Train: 396/426 (93.0%)
Test: 137/143 (95.8%)

c = 0.25
sum(beta**2) =  1.159763781457247
Train: 392/426 (92.0%)
Test: 135/143 (94.4%)

c = 1.0
sum(beta**2) =  0.21983095016705734
Train: 390/426 (91.5%)
Test: 134/143 (93.7%)

c = 4.0
sum(beta**2) =  0.08526863018405177
Train: 390/426 (91.5%)
Test: 134/143 (93.7%)

c = 16.0
sum(beta**2) =  0.06240644243334323
Train: 390/426 (91.5%)
Test: 134/143 (93.7%)

c = 64.0
sum(beta**2) =  0.043885815938907356
Train: 388/426 (91.1%)
Test: 134/143 (93.7%)

c = 256.0
sum(beta**2) =  0.016109390288629684
Train: 387/426 (90.8%)
Test: 136/143 (95.1%)



In [19]:
from sklearn import preprocessing

def inputs(t):
    return t.drop('malignant', axis=1).values

scaler = preprocessing.StandardScaler().fit(inputs(train))

def scaled_features(t):
    return scaler.transform(inputs(t)).T

search_for_c(scaled_features)

c = 0.0009765625
sum(beta**2) =  35.8486017048037
Train: 423/426 (99.3%)
Test: 138/143 (96.5%)

c = 0.00390625
sum(beta**2) =  31.318273815172542
Train: 423/426 (99.3%)
Test: 138/143 (96.5%)

c = 0.015625
sum(beta**2) =  20.747956964144077
Train: 423/426 (99.3%)
Test: 139/143 (97.2%)

c = 0.0625
sum(beta**2) =  9.935156093274612
Train: 421/426 (98.8%)
Test: 141/143 (98.6%)

c = 0.25
sum(beta**2) =  4.279626098449614
Train: 419/426 (98.4%)
Test: 141/143 (98.6%)

c = 1.0
sum(beta**2) =  1.717220435886093
Train: 414/426 (97.2%)
Test: 140/143 (97.9%)

c = 4.0
sum(beta**2) =  0.6094833404635616
Train: 412/426 (96.7%)
Test: 139/143 (97.2%)

c = 16.0
sum(beta**2) =  0.17553643009069883
Train: 405/426 (95.1%)
Test: 138/143 (96.5%)

c = 64.0
sum(beta**2) =  0.03500234439319296
Train: 403/426 (94.6%)
Test: 136/143 (95.1%)

c = 256.0
sum(beta**2) =  0.0041317425176505785
Train: 399/426 (93.7%)
Test: 134/143 (93.7%)



In [20]:
model = LogisticRegression(C=4, solver='lbfgs')
model.fit(scaled_features(train).T, y_train)
y_hat = model.predict(scaled_features(test).T)
print_ratio(sum(y_hat == y_test), len(y_test))

138/143 (96.5%)


### Multiclass classification

\begin{align*}
P(Y=y|X) &= \frac{\exp(X^T\beta_{y})}{\sum_{z=0}^d \exp(X^T\beta_z)} \\[10pt]
L(\beta_0,\dots,\beta_d, x_i, y_i) &= - \log \frac{\exp(x_i^T\beta_{y_i})}{\sum_{z=0}^d \exp(x_i^T\beta_z)} \\[10pt]
\frac{\partial}{\partial \beta_w} L(\beta_0,\dots,\beta_d, x_i, y_i) &= -\left(1[w=y_i] - \frac{\exp(x_i^T\beta_w)}{\sum_{z=0}^d \exp(x_i^T\beta_z)}\right) x_i  \\[10pt]
1[w=y_i] &= \begin{cases}
1 & \text{if}\ w=y_i \\
0 & \text{otherwise}
\end{cases}
\end{align*}


In [21]:
data_dict = sklearn.datasets.load_iris()
x = pd.DataFrame(data_dict['data'], columns=data_dict['feature_names'])
y = data_dict['target']
x.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [22]:
np.unique(y, return_counts=True)

(array([0, 1, 2]), array([50, 50, 50]))